# W7 Activity

In [1]:
# Setup
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

## Exercise 1

In [5]:
img = cv.imread('lena.jfif', 0) #grayscale

# Histogram Equalization
eq = cv.equalizeHist(img)

# 2nd Histogram Equalization
dst = cv.equalizeHist(eq)

cv.imshow('result', np.hstack((img, eq, dst)))
cv.waitKey()
cv.destroyAllWindows()

There is no changes between 2 images.
The value of the histogram equalization will not change while it is multiplied by itself.

## Exercise 2

### part a: different kernel size

In [23]:
img = cv.imread('electronic.jfif', 0)

laplacian1 = cv.Laplacian(img, cv.CV_64F, ksize = 1)
laplacian1_8u = np.uint8(np.absolute(laplacian1))

laplacian2 = cv.Laplacian(img, cv.CV_64F, ksize = 3)
laplacian2_8u = np.uint8(np.absolute(laplacian2))

laplacian3 = cv.Laplacian(img, cv.CV_64F, ksize = 5)
laplacian3_8u = np.uint8(np.absolute(laplacian3))

laplacian4 = cv.Laplacian(img, cv.CV_64F, ksize = 7)
laplacian4_8u = np.uint8(np.absolute(laplacian4))

cv.imshow('result', np.hstack((laplacian1_8u, laplacian2_8u, laplacian3_8u, laplacian4_8u)))

cv.waitKey(0)
cv.destroyAllWindows()

The smaller kernel size is preferred.

### part b: sobel with combined x and y with and without Gaussian blur

In [31]:
img = cv.imread('electronic.jfif', 0)
img_blur = cv.GaussianBlur(img, (5, 5), 0)

sobelx_normal = cv.Sobel(img, cv.CV_64F, 1, 0, ksize = 3)
sobelx_8u_normal = np.uint8(np.absolute(sobelx_normal))

sobely_normal = cv.Sobel(img, cv.CV_64F, 0, 1, ksize = 3)
sobely_8u_normal = np.uint8(np.absolute(sobely_normal))
    
edges_normal = sobelx_8u_normal + sobely_8u_normal

sobelx = cv.Sobel(img, cv.CV_64F, 1, 0, ksize = 3)
sobelx_8u = np.uint8(np.absolute(sobelx))

sobely = cv.Sobel(img, cv.CV_64F, 0, 1, ksize = 3)
sobely_8u = np.uint8(np.absolute(sobely))
    
edges = sobelx_8u + sobely_8u

cv.imshow('result', np.hstack((edges_normal, edges)))

cv.waitKey(0)
cv.destroyAllWindows()

### part c: Laplacian of Gaussian (LoG)

In [29]:
img = cv.imread('electronic.jfif', 0)
img_blur = cv.GaussianBlur(img, (5, 5), 0)

laplacian = cv.Laplacian(img_blur, cv.CV_64F, ksize = 1)
laplacian_8u = np.uint8(np.absolute(laplacian))

cv.imshow('result', laplacian_8u)

cv.waitKey(0)
cv.destroyAllWindows()

In my opinion, the Laplacian of Gaussian pathway with smaller kernel size (1) would be optimal. As Laplacian filter is susceptible to noise, after removing the noise by Gaussian blur, the result seems satisfactory compared to the second path with Gaussian blur. 

## Exercise 3

In [34]:
img = cv.imread('pineapple.jfif', 0)
img_blur = cv.GaussianBlur(img, (5, 5), 0)

# sobel

sobelx = cv.Sobel(img_blur, cv.CV_64F, 1, 0, ksize = 3)
sobelx_8u = np.uint8(np.absolute(sobelx))

sobely = cv.Sobel(img_blur, cv.CV_64F, 0, 1, ksize = 3)
sobely_8u = np.uint8(np.absolute(sobely))
    
edges = sobelx_8u + sobely_8u

# Laplacian
laplacian = cv.Laplacian(img_blur, cv.CV_64F, ksize = 3)
laplacian_8u = np.uint8(np.absolute(laplacian))

# Prewitt
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])

img_prewittx = cv.filter2D(img_blur, -1, kernelx)
img_prewitty = cv.filter2D(img_blur, -1, kernely)

img_prewitt = img_prewittx + img_prewitty

# Scharr
sX = cv.Scharr(img_blur, cv.CV_64F, 1, 0, 3)
sX = np.uint8(np.absolute(sX))
sY = cv.Scharr(img_blur, cv.CV_64F, 0, 1, 3)
sY = np.uint8(np.absolute(sY))
s = sX + sY

# Canny
edges_canny = cv.Canny(img_blur, 100, 200)

cv.imshow('result', np.hstack((edges, laplacian_8u, img_prewitt, s, edges_canny)))

cv.waitKey(0)
cv.destroyAllWindows()

Based on the result shown, the most preferable edge detection method would be Canny. Since it has more noise immunity than other methods and we can establish inferior and superior threshold for it.

The worst one would be the Scharr method by referring to the result as it is very sensitive to noise.

## Exercise 4

In [39]:
img = cv.imread('electronic.jfif')

# white color boundaries [R, G, B]
lower = [160, 160, 160]
upper = [255, 255, 255]

# create NumPy arrays from the boundaries
lower = np.array(lower, dtype = "uint8")
upper = np.array(upper, dtype = "uint8")

# find the colors within the specified boundaries and apply the mask
mask = cv.inRange(img, lower, upper)
output = cv.bitwise_and(img, img, mask = mask)

ret,thresh = cv.threshold(mask, 40, 255, 0)

contours, hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

if len(contours) != 0:
    # draw in blue the contours that were founded
    cv.drawContours(output, contours, -1, 255, 3)

    # find the biggest countour (c) by the area
    c = max(contours, key = cv.contourArea)
    x, y, w, h = cv.boundingRect(c)

    # draw the biggest contour (c) in green
    cv.rectangle(img,(x,y),(x + w, y + h), (0, 255, 0), 2)

# show the images
cv.imshow("Result", np.hstack([img, output]))

cv.waitKey(0)
cv.destroyAllWindows()